In [1]:
### 한글 폰트 설정
import matplotlib
from matplotlib import font_manager, rc
import matplotlib.pyplot as plt
import platform
path = "C:/Windows/Fonts/malgun.ttf"
if platform.system() == "Windows":
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
elif platform.system()=="Darwin":
    rc('font', family='AppleGothic')
else:
    print("Unknown System")
matplotlib.rcParams['axes.unicode_minus'] = False
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression

In [3]:
import pandas as pd

train = pd.read_csv('train_df_errno.csv')
test = pd.read_csv('test_df.csv')
sub = pd.read_csv('sample_submission.csv')
age = pd.read_csv('age_gender_info.csv')

train.shape, test.shape, sub.shape, age.shape

((2896, 15), (1008, 14), (150, 2), (16, 23))

In [4]:
train.columns

Index(['단지코드', '총세대수', '임대건물구분', '지역', '공급유형', '전용면적', '전용면적별세대수', '공가수',
       '자격유형', '임대보증금', '임대료', '10분내지하철수', '10분내버스정류장수', '단지내주차면수', '등록차량수'],
      dtype='object')

In [5]:
all_df = pd.concat([train, test], join='inner')
all_df

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,A,9216000,82940,0.0,3.0,624.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,A,12672000,107130,0.0,3.0,624.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,A,12672000,107130,0.0,3.0,624.0
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,A,18433000,149760,0.0,3.0,624.0
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,A,18433000,149760,0.0,3.0,624.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,L,-,-,0.0,1.0,467.0
1004,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,H,6872000,106400,0.0,2.0,300.0
1005,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,H,6872000,106400,0.0,2.0,300.0
1006,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,H,13410000,144600,0.0,2.0,300.0


In [6]:
all_df.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            2
임대보증금         749
임대료           749
10분내지하철수      249
10분내버스정류장수      4
단지내주차면수         0
dtype: int64

In [7]:
all_df.loc[all_df['자격유형'].isnull()]

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
196,C2411,962,아파트,경상남도,국민임대,46.90,240,25.0,NaN,71950000,37470,0.0,2.0,840.0
258,C2253,1161,아파트,강원도,영구임대,26.37,745,0.0,NaN,2249000,44770,0.0,2.0,173.0


In [8]:
all_df.loc[ 196, "자격유형"] = 'A'
all_df.loc[ 258, "자격유형"] = 'C'

In [9]:
mapping = { 'A':1, 'B':2, 'C':3, 'D':4, 'E':5,
            'F':6, 'G':7, 'H':8, 'I':9, 'J':10,
            'K':11, 'L':12, 'M':13, 'N':14, 'O':15  }
all_df['자격유형'] =all_df['자격유형'].map(mapping).astype(int)

In [10]:
all_df.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,1,9216000,82940,0.0,3.0,624.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,1,12672000,107130,0.0,3.0,624.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,1,12672000,107130,0.0,3.0,624.0
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,1,18433000,149760,0.0,3.0,624.0
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,1,18433000,149760,0.0,3.0,624.0


In [11]:
all_df.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         749
임대료           749
10분내지하철수      249
10분내버스정류장수      4
단지내주차면수         0
dtype: int64

In [12]:
all_df.corr()['10분내버스정류장수']

총세대수         -0.002586
전용면적          0.002303
전용면적별세대수      0.040635
공가수           0.038906
자격유형         -0.014581
10분내지하철수      0.058901
10분내버스정류장수    1.000000
단지내주차면수       0.097617
Name: 10분내버스정류장수, dtype: float64

In [13]:
grouped = train.groupby(['임대건물구분','지역'])
group1 = grouped.get_group(('아파트', '경상남도'))
group1['10분내버스정류장수'].mean()

3.996268656716418

In [14]:
grouped = train.groupby(['임대건물구분', '지역'])
group1 = grouped.get_group( ('아파트', '경상남도')  )
val = group1['10분내버스정류장수'].mean()
val

3.996268656716418

In [15]:
# 데이터 확인 후,
all_df.loc[ all_df['10분내버스정류장수'].isnull(), "10분내버스정류장수"] = val

In [16]:
all_df.isnull().sum()

단지코드            0
총세대수            0
임대건물구분          0
지역              0
공급유형            0
전용면적            0
전용면적별세대수        0
공가수             0
자격유형            0
임대보증금         749
임대료           749
10분내지하철수      249
10분내버스정류장수      0
단지내주차면수         0
dtype: int64

In [17]:
all_df.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,1,9216000,82940,0.0,3.0,624.0
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,1,12672000,107130,0.0,3.0,624.0
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,1,12672000,107130,0.0,3.0,624.0
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,1,18433000,149760,0.0,3.0,624.0
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,1,18433000,149760,0.0,3.0,624.0


In [18]:
gubun1 = {'아파트':1, '상가':2}
gubun2 = {'경상남도':1, '대전광역시':2, '경기도':3, '전라북도':4, '강원도':5,
          '광주광역시':6, '충청남도':7, '부산광역시':8, '제주특별자치도':9, '울산광역시':10,
          '충청북도':11, '전라남도':12, '경상북도':13, '대구광역시':14, '서울특별시':15, '세종특별자치시':16}
gubun3 = {'국민임대':1, '공공임대(50년)':2, '영구임대':3, '임대상가':4, '공공임대(10년)':5,
          '공공임대(분납)':6, '장기전세':7, '공공분양':8, '행복주택':9, '공공임대(5년)':10}
all_df['임대건물구분_lbl'] = all_df['임대건물구분'].map(gubun1)
all_df['지역_lbl'] = all_df['지역'].map(gubun2)
all_df['공급유형_lbl'] = all_df['공급유형'].map(gubun3)
all_df

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,임대건물구분_lbl,지역_lbl,공급유형_lbl
0,C2515,545,아파트,경상남도,국민임대,33.48,276,17.0,1,9216000,82940,0.0,3.0,624.0,1,1,1
1,C2515,545,아파트,경상남도,국민임대,39.60,60,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1
2,C2515,545,아파트,경상남도,국민임대,39.60,20,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1
3,C2515,545,아파트,경상남도,국민임대,46.90,38,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1
4,C2515,545,아파트,경상남도,국민임대,46.90,19,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,C1267,675,아파트,경상남도,행복주택,36.77,126,38.0,12,-,-,0.0,1.0,467.0,1,1,9
1004,C2189,382,아파트,전라북도,국민임대,29.19,96,45.0,8,6872000,106400,0.0,2.0,300.0,1,4,1
1005,C2189,382,아파트,전라북도,국민임대,29.19,20,45.0,8,6872000,106400,0.0,2.0,300.0,1,4,1
1006,C2189,382,아파트,전라북도,국민임대,39.45,202,45.0,8,13410000,144600,0.0,2.0,300.0,1,4,1


In [19]:
all_df['단지코드'] = all_df['단지코드'].astype("category")
all_df['단지코드_lbl'] = all_df['단지코드'].cat.codes

In [20]:
all_df_last = all_df.drop(['임대건물구분', '지역', '공급유형'] , axis=1)
all_df_last

,단지코드,총세대수,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,임대건물구분_lbl,지역_lbl,공급유형_lbl,단지코드_lbl
0,C2515,545,33.48,276,17.0,1,9216000,82940,0.0,3.0,624.0,1,1,1,492
1,C2515,545,39.60,60,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1,492
2,C2515,545,39.60,20,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1,492
3,C2515,545,46.90,38,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1,492
4,C2515,545,46.90,19,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1,492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003,C1267,675,36.77,126,38.0,12,-,-,0.0,1.0,467.0,1,1,9,86
1004,C2189,382,29.19,96,45.0,8,6872000,106400,0.0,2.0,300.0,1,4,1,381
1005,C2189,382,29.19,20,45.0,8,6872000,106400,0.0,2.0,300.0,1,4,1,381
1006,C2189,382,39.45,202,45.0,8,13410000,144600,0.0,2.0,300.0,1,4,1,381


In [21]:
train_df = all_df_last.iloc[0:2896,:]
test_df = all_df_last.iloc[2896:,:]

train_df.shape, test_df.shape

train_df = pd.concat([train_df, train['등록차량수'] ], axis=1)
train_df

,단지코드,총세대수,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,10분내지하철수,10분내버스정류장수,단지내주차면수,임대건물구분_lbl,지역_lbl,공급유형_lbl,단지코드_lbl,등록차량수
0,C2515,545,33.48,276,17.0,1,9216000,82940,0.0,3.0,624.0,1,1,1,492,205.0
1,C2515,545,39.60,60,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1,492,205.0
2,C2515,545,39.60,20,17.0,1,12672000,107130,0.0,3.0,624.0,1,1,1,492,205.0
3,C2515,545,46.90,38,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1,492,205.0
4,C2515,545,46.90,19,17.0,1,18433000,149760,0.0,3.0,624.0,1,1,1,492,205.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2891,C2532,239,49.20,19,7.0,1,11346000,116090,0.0,1.0,166.0,1,5,1,501,146.0
2892,C2532,239,51.08,34,7.0,1,14005000,142310,0.0,1.0,166.0,1,5,1,501,146.0
2893,C2532,239,51.73,34,7.0,1,14005000,142310,0.0,1.0,166.0,1,5,1,501,146.0
2894,C2532,239,51.96,114,7.0,1,14005000,142310,0.0,1.0,166.0,1,5,1,501,146.0


In [22]:
from sklearn.model_selection import train_test_split

In [23]:
train_df.corr()['등록차량수']

총세대수          0.333513
전용면적          0.112717
전용면적별세대수      0.250513
공가수           0.118910
자격유형         -0.154034
10분내지하철수     -0.107308
10분내버스정류장수    0.104200
단지내주차면수       0.861338
임대건물구분_lbl   -0.449130
지역_lbl        0.060674
공급유형_lbl     -0.137277
단지코드_lbl     -0.062077
등록차량수         1.000000
Name: 등록차량수, dtype: float64

In [24]:
sel = [ '총세대수', '전용면적', '전용면적별세대수', '공가수', '자격유형', '10분내버스정류장수', '단지내주차면수',
       '임대건물구분_lbl', '지역_lbl', '공급유형_lbl', '단지코드_lbl']
X = train_df[sel]
y = train_df['등록차량수']
test_X = test_df[sel]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state=0)

In [25]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
#import xgboost as xgb
#import lightgbm as lgb

In [26]:
model = RandomForestRegressor(n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(X_test)
print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

학습(score) : 0.9992596072209938
테스트(score) : 0.9974334764078387


In [27]:
model = GradientBoostingRegressor()  # RF보다 설명력 좋음
model.fit(X_train, y_train)
pred = model.predict(X_test)
print("학습(score) :", model.score(X_train, y_train) ) # 결정계수
print("테스트(score) :", model.score(X_test, y_test) ) # 결정계수

학습(score) : 0.9616060362116032
테스트(score) : 0.9570190958624976


In [28]:
import time

In [29]:
now_time = time.time()

model_RF = RandomForestRegressor(n_estimators = 1000, random_state=30, n_jobs=-1)  # n_jobs는 시간함수와 함께 사용하여 걸린 시간 측정
model_RF.fit(X_train, y_train)
score = cross_val_score(model_RF, X_train, y_train,
                        cv=5, scoring="neg_mean_absolute_error") # neg_mean_squared_error
m_score = np.abs(score.mean())
print("RandomForestRegressor Score : {}".format(m_score))  # 점수

pro_time = time.time() - now_time
print(pro_time)  # 걸린 시간

RandomForestRegressor Score : 16.33284789124951
133.13594961166382


In [30]:
now_time = time.time()

model_GB = GradientBoostingRegressor(random_state=30)
model_GB.fit(X_train, y_train)
score = cross_val_score(model_GB, X_train, y_train,
                        cv=5, scoring="neg_mean_absolute_error") # neg_mean_squared_error
m_score = np.abs(score.mean())
print("GradientBoostingRegressor Score : {}".format(m_score))  # 점수

pro_time = time.time() - now_time
print(pro_time)  # 걸린 시간

GradientBoostingRegressor Score : 69.55727356371403
14.342217683792114


In [31]:
hyperparameters = {'boosting_type': 'gbdt',
                   'colsample_bytree': 0.7250136792694301,
                   'is_unbalance': False,
                   'learning_rate': 0.013227664889528229,
                   'min_child_samples': 20,
                   'num_leaves': 56,
                   'reg_alpha': 0.7543896477745794,
                   'reg_lambda': 0.07152751159655985,
                   'subsample_for_bin': 240000,
                   'subsample': 0.5233384321711397,
                   'n_estimators': 1093}

In [33]:
import lightgbm as lgb

ow_time = time.time()
m_lgbm1 = lgb.LGBMRegressor(**hyperparameters)
m_lgbm1.fit(X_train, y_train)
score = cross_val_score(m_lgbm1, X_train, y_train,
                        cv=5, scoring="neg_mean_absolute_error")
m_score = np.abs(score.mean())  # 절대값
pro_time = time.time() - now_time

print(pro_time)  # 걸린 시간
print("LightGBM Score : {}".format(m_score))  # 점수

349.2534210681915
LightGBM Score : 20.179727490825435


In [37]:
"""
hyperparameters = {'boosting_type': 'gbdt',
                   'colsample_bytree': 0.7250136792694301,
                   'is_unbalance': False,
                   'learning_rate': 0.013227664889528229,
                   'min_child_samples': 20,
                   'num_leaves': 56,
                   'reg_alpha': 0.7543896477745794,
                   'reg_lambda': 0.07152751159655985,
                   'subsample_for_bin': 240000,
                   'subsample': 0.5233384321711397,
                   'n_estimators': 1093}
"""

"\nhyperparameters = {'boosting_type': 'gbdt',\n                   'colsample_bytree': 0.7250136792694301,\n                   'is_unbalance': False,\n                   'learning_rate': 0.013227664889528229,\n                   'min_child_samples': 20,\n                   'num_leaves': 56,\n                   'reg_alpha': 0.7543896477745794,\n                   'reg_lambda': 0.07152751159655985,\n                   'subsample_for_bin': 240000,\n                   'subsample': 0.5233384321711397,\n                   'n_estimators': 1093}\n"

* LightGBM Score : 20.179727490825435
    * learning_rate : 0.013227664889528229 
    * learning_rate
    * n_estimators
    * learning_rate